In [ ]:
!pip install transformers faiss-cpu scikit-learn --quiet

In [ ]:

from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.feature_extraction.text import TfidfVectorizer
import torch

knowledge_base = [
    "Artificial Intelligence (AI) is the simulation of human intelligence in machines.",
    "Machine Learning (ML) is a subset of AI that focuses on training algorithms to learn from data.",
    "Transformers are a type of neural network architecture used in natural language processing.",
    "Retrieval-Augmented Generation (RAG) enhances chatbots by retrieving context-relevant knowledge before generating responses.",
    "TF-IDF stands for Term Frequency-Inverse Document Frequency, and it is used for information retrieval.",
    "GPT models are pre-trained language models designed to generate human-like text.",
    "RAG combines retrieval and generation to create a smarter AI system."
]

vectorizer = TfidfVectorizer()
knowledge_vectors = vectorizer.fit_transform(knowledge_base)


model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


def retrieve_context(query, top_k=1):
    """Retrieve the most relevant knowledge from the knowledge base."""
    query_vector = vectorizer.transform([query])
    scores = (knowledge_vectors * query_vector.T).toarray().flatten()
    top_indices = scores.argsort()[-top_k:][::-1]
    return " ".join([knowledge_base[i] for i in top_indices])


def chatbot_response(user_input):
    context = retrieve_context(user_input)
    prompt = f"Context: {context}\n\nUser: {user_input}\nBot:"
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    outputs = model.generate(
        inputs,
        max_length=300,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("Bot:")[-1].strip()

print("Chatbot: Hi there! Ask me anything about AI, ML, or related topics. (Type 'exit' to quit)")
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Chatbot: Goodbye!")
        break
    try:
        response = chatbot_response(user_input)
        print(f"Chatbot: {response}")
    except Exception as e:
        print(f"An error occurred: {e}")


Chatbot: Hi there! Ask me anything about AI, ML, or related topics. (Type 'exit' to quit)
You: what is GPT
Chatbot: GPT is a language model that is designed to generate human-like text.
You: exit
Chatbot: Goodbye!
